In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import cv2 as cv

import downloader as d 
import video as v

In [14]:
from typing import Tuple, Union
from numpy.typing import NDArray
from dataclasses import dataclass
from collections import deque

In [6]:
from importlib import reload

In [8]:
reload(v)

<module 'video' from '/Users/nadina/Documents/codeup/epilator/video.py'>

In [33]:
def get_video(path: str) -> Tuple[NDArray, int]:
    vid = v.Video(path)
    fps = vid.video.get(cv.CAP_PROP_FPS)
    vid = np.asarray(vid)
    video = np.fromiter(vid, np.ndarray)
    return np.stack(video), int(round(fps))

def get_vid_df(path: str) -> pd.DataFrame:
    vid,fps = get_video(path)
    frames = vid.shape[0]
    height = vid.shape[1]
    width = vid.shape[2]
    df = pd.DataFrame(vid.reshape((-1,3)))
    df['frame'] = df.index // (width * height)
    df['x'] = df.index % width
    df['y'] = df.index // width % height
    df = df.set_index(['frame','y','x']).rename(columns={
        0:'hue',
        1:'lightness',
        2:'saturation',
    })
    df.attrs['fps'] = fps
    return df

In [27]:
vid = v.Video('videos/Banned Pokemon Seizure Scene.mp4')

In [29]:
vid1 = cv.VideoCapture('videos/Banned Pokemon Seizure Scene.mp4')

In [36]:
vid2 = vid1.read()

AttributeError: module 'cv2' has no attribute 'GetMat'

In [30]:
type(vid1)

cv2.VideoCapture

In [34]:
get_video('videos/Banned Pokemon Seizure Scene.mp4')

TypeError: iteration over a 0-d array

In [9]:
df = v.get_vid_df('videos/Banned Pokemon Seizure Scene.mp4')

ValueError: cannot create object arrays from iterator